## Classifying Handwritten Digits

In [ ]:
import matplotlib.pyplot as plt
import sklearn.datasets
import sklearn.model_selection
import sklearn.metrics

We use the toy digit dataset provided by scikit-learn.  

(We will use the full MNIST dataset shortly, one of the classic initial problems for budding machine-learning practicioners.)

In [ ]:
d = sklearn.datasets.load_digits()

In [ ]:
print(d.DESCR)

In [ ]:
x = d.data
y = d.target

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
x[0]

In [ ]:
y[0]

The samples consist of 64 features, one for each pixel value of an 8x8 image array.  We can reshape the sample into an 8x8 array in order to visualize it.

In [ ]:
sample = x[4].reshape(8,8)
plt.imshow(sample, cmap='binary')

In [ ]:
for i in range(100):
    plt.subplot(10,10,i+1)
    sample = x[i].reshape(8,8)
    plt.imshow(sample, cmap='binary')

## Stochastic Gradient Descent

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd_classifier = SGDClassifier(random_state=42)

In [ ]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(
        x, y, test_size=0.2, random_state=42)

One catch to watch out for in splitting up your data into training and test sets:  stratification.

Let's say you have a dataset that has 90% cat images and 10% dog images.  If you split your data and end up with 99% cats in your training data and 1% dogs, you'll be training your model on an unrepresentative sample.  (Sampling issues like this can be much more consequential and damaging than distinguishing cats from dogs!)

In [ ]:
plt.hist(y, width=0.5)

In [ ]:
plt.hist(y_train, width=0.5)

Here the difference in percentages is noticeable but not too significant by eye.  Nevertheless, we can stratify our split properly by including the "stratify" parameter and assigning it our target variable.

In [ ]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(
        x, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
plt.hist(y, width=0.5)

In [ ]:
plt.hist(y_train, width=0.5)

In [ ]:
sgd_classifier.fit(x_train, y_train)

In [ ]:
sgd_classifier.predict(x_train[[7]])

In [ ]:
y_train[7]

In [ ]:
y_pred = sgd_classifier.predict(x_test)

In [ ]:
cm = sklearn.metrics.confusion_matrix(y_test, y_pred)
cm

In contrast with binary classification, calculating precision and recall (and etc) for multi-class classification problems can be computed in slightly different ways depending on how one does averaging. 

A macro-average will compute the metric independently for each class and then take the average (hence treating all classes equally), whereas a micro-average will aggregate the contributions of all classes to compute the average metric. 

In a multi-class classification setup, micro-average is preferable if you suspect there might be class imbalance (i.e you may have many more examples of one class than of other classes).

In [ ]:
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, y_pred):.2%}")
print(f"Precision: {sklearn.metrics.precision_score(y_test, y_pred, average='micro'):.2%}")
print(f"Recall: {sklearn.metrics.recall_score(y_test, y_pred, average='micro'):.2%}")

In [ ]:
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, y_pred):.2%}")
print(f"Precision: {sklearn.metrics.precision_score(y_test, y_pred, average='macro'):.2%}")
print(f"Recall: {sklearn.metrics.recall_score(y_test, y_pred, average='macro'):.2%}")

## Logistic Regression

In [ ]:
import sklearn.linear_model
lr_classifier = sklearn.linear_model.LogisticRegression()

In [ ]:
lr_classifier.fit(x_train, y_train)

It will not be uncommon for you to run into scenarios in which you encounter errors when trying to train models.

In such cases, they can be fruitful opportunities to consult the documentation and learn more about various training options.

Here, the error message gives us clues about potentially insightful documentation.

To fast-forward, it will be useful here for Logistic Regression if we rescale our sample data from being integer values over [0:16] to being continuous values scaled to have a normal distribution of values -> the sklearn StandardScaler will rescale the features to have 0 mean and unit variance.

In [ ]:
import sklearn.preprocessing

In [ ]:
scaler = sklearn.preprocessing.StandardScaler()

In [ ]:
x_scaled = scaler.fit_transform(x_train)

In [ ]:
x_scaled[0]

Here's the difference in image between original and rescaled.

In [ ]:
sample = x_train[7].reshape(8,8)
plt.imshow(sample,cmap='binary')

In [ ]:
sample = x_scaled[7].reshape(8,8)
plt.imshow(sample,cmap='binary')

In [ ]:
for i in range(100):
    plt.subplot(10,10,i+1)
    sample = x_scaled[i].reshape(8,8)
    plt.imshow(sample, cmap='binary')

In [ ]:
lr_classifier.fit(x_scaled, y_train)

In [ ]:
lr_classifier.predict(x_scaled[[7]])

In [ ]:
y_train[7]

Our classifier was trained on scaled data, so we must scale any new data similarly (though we only need to do the transform now, not the fit.)

In [ ]:
x_test_scaled = scaler.transform(x_test)

In [ ]:
y_pred = lr_classifier.predict(x_test_scaled)

In [ ]:
cm = sklearn.metrics.confusion_matrix(y_test, y_pred)
cm

In [ ]:
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, y_pred):.2%}")

## Wait.... What does SGDClassifier optimize?

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html

In [ ]:
sgd_classifier_logloss = SGDClassifier(loss='log_loss',random_state=42)

In [ ]:
sgd_classifier_logloss.fit(x_scaled, y_train)

In [ ]:
sgd_classifier_logloss.predict(x_scaled[[7]])

In [ ]:
y_train[7]

In [ ]:
y_pred = sgd_classifier_logloss.predict(x_test_scaled)

In [ ]:
cm = sklearn.metrics.confusion_matrix(y_test, y_pred)
cm

In [ ]:
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, y_pred):.2%}")

# Decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree_clf = DecisionTreeClassifier()

In [ ]:
tree_clf.fit(x_train, y_train)

In [ ]:
tree_clf.classes_

In [ ]:
y_pred = tree_clf.predict(x_test)

In [ ]:
cm = sklearn.metrics.confusion_matrix(y_test, y_pred)
cm

In [ ]:
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, y_pred):.2%}")

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier()

In [ ]:
rf_clf.fit(x_train, y_train)

In [ ]:
rf_clf.classes_

In [ ]:
y_pred = rf_clf.predict(x_test)

In [ ]:
cm = sklearn.metrics.confusion_matrix(y_test, y_pred)
cm

In [ ]:
scaler = sklearn.preprocessing.StandardScaler()
scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, sgd_classifier.predict(x_test)):.2%}")
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, lr_classifier.predict(x_test_scaled)):.2%}")
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, tree_clf.predict(x_test)):.2%}")
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, rf_clf.predict(x_test)):.2%}")

Can we improve the Random Forest accuracy?

Actually, what parameters does it currently have?

In [ ]:
rf_clf.get_params()

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
cv_grid = GridSearchCV(RandomForestClassifier(n_jobs=-1,random_state=42),
                       param_grid = {
                           'max_depth' : [None,10,20],
                           'n_estimators' : [50,100,200],
                           'max_leaf_nodes' : [None,5,10]
                       })
cv_grid.fit(x_train, y_train)
cv_grid.best_params_

In [ ]:
y_pred = cv_grid.predict(x_test)

In [ ]:
cm = sklearn.metrics.confusion_matrix(y_test, y_pred)
cm

In [ ]:
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, y_pred):.2%}")

In [ ]:
rf_clf.feature_importances_

In [ ]:
plt.imshow(rf_clf.feature_importances_.reshape(8,8),
           cmap='binary')

## Extra section on classification performance

In classification there is a trade-off between optimizing precision and optimizing recall.

The following will allow you to quantify and visualize that trade-off, as well as make plots of the ROC curves (Receiver Operating Characteristic).

ROC is useful for binary classification, when you have a strict population of false-negatives, true-positives, and etc.  Precision and recall are also more easily conceptualized in binary classification, when there is no ambiguity about handling multi-class classification.  Therefore, the below does classification for the classes Fives and Not-Fives.  (You can easily check for other numbers too by changing the 5 to, say, 9).

In [ ]:
y_five = (y == 5)

In [ ]:
y_five

In [ ]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(
        x, y_five, test_size=0.2, random_state=42)

## Another "wait..."

Five vs not-five is taking us into smaller sample sizes for the class we're trying to predict.

Let's see what percentages of the training set and test set are 5s

In [ ]:
import numpy as np

In [ ]:
np.sum(y_five) / y_five.shape

In [ ]:
np.sum(y_test) / y_test.shape

In [ ]:
np.sum(y_train) / y_train.shape

Probably better to make sure that the sample is properly stratified.

In [ ]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(
        x, y_five, test_size=0.2, random_state=42, stratify=y_five)

In [ ]:
import numpy as np

In [ ]:
np.sum(y_five) / y_five.shape

In [ ]:
np.sum(y_test) / y_test.shape

In [ ]:
np.sum(y_train) / y_train.shape

Those 10% numbers are looking better.

In [ ]:
# Remember:  These are for binary classification 
# rather than the full multicategory classification above

# Stochastic Gradient Descent
sgd_classifier = sklearn.linear_model.SGDClassifier(random_state=42)
sgd_classifier.fit(x_train, y_train)
sgd_y_pred = sgd_classifier.predict(x_test)
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, sgd_y_pred):.2%}")
print(f"SGD Precision: {sklearn.metrics.precision_score(y_test, sgd_y_pred):.2%}")
print(f"SGD Recall: {sklearn.metrics.recall_score(y_test, sgd_y_pred):.2%}")
cm = sklearn.metrics.confusion_matrix(y_test, sgd_y_pred)
print(cm)

# Logistic Regression
lr_classifier = sklearn.linear_model.LogisticRegression(random_state=42)
lr_scaler = sklearn.preprocessing.StandardScaler()
x_train_scaled = lr_scaler.fit_transform(x_train)
x_test_scaled = lr_scaler.transform(x_test)
lr_classifier.fit(x_train_scaled, y_train)
lr_y_pred = lr_classifier.predict(x_test_scaled)
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, lr_y_pred):.2%}")
print(f"LR Precision: {sklearn.metrics.precision_score(y_test, lr_y_pred):.2%}")
print(f"LR Recall: {sklearn.metrics.recall_score(y_test, lr_y_pred):.2%}")
cm = sklearn.metrics.confusion_matrix(y_test, lr_y_pred)
print(cm)

# Decision Tree
tree_classifier = sklearn.tree.DecisionTreeClassifier()
tree_classifier.fit(x_train, y_train)
tree_y_pred = tree_classifier.predict(x_test)
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, tree_y_pred):.2%}")
print(f"Tree Precision: {sklearn.metrics.precision_score(y_test, tree_y_pred):.2%}")
print(f"Tree Recall: {sklearn.metrics.recall_score(y_test, tree_y_pred):.2%}")
cm = sklearn.metrics.confusion_matrix(y_test, tree_y_pred)
print(cm)

# Random Forest
rf_classifier = sklearn.ensemble.RandomForestClassifier()
rf_classifier.fit(x_train, y_train)
rf_y_pred = rf_classifier.predict(x_test)
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, rf_y_pred):.2%}")
print(f"RF Precision: {sklearn.metrics.precision_score(y_test, rf_y_pred):.2%}")
print(f"RF Recall: {sklearn.metrics.recall_score(y_test, rf_y_pred):.2%}")
cm = sklearn.metrics.confusion_matrix(y_test, rf_y_pred)
print(cm)


In [ ]:
sgd_y_scores = sklearn.model_selection.cross_val_predict(sgd_classifier, x_train, y_train, cv=3,
                             method="decision_function")

In [ ]:
sgd_y_scores

In [ ]:
print(sum(sgd_y_scores > 0))
print(sum(sgd_y_scores < 0))

In [ ]:
print(sum(sgd_y_scores > 0) / len(sgd_y_scores))
print(sum(sgd_y_scores < 0) / len(sgd_y_scores))

In [ ]:
sgd_precisions, sgd_recalls, sgd_thresholds = sklearn.metrics.precision_recall_curve(y_train, sgd_y_scores)

plt.figure(figsize=(8, 4))
plt.plot(sgd_thresholds, sgd_precisions[:-1], "b--", label="SGD Precision", linewidth=2)
plt.plot(sgd_thresholds, sgd_recalls[:-1], "g-", label="SGD Recall", linewidth=2)
plt.axis([-3000, 3000, 0, 1.1])
plt.xlabel("Threshold")
plt.legend()

In [ ]:
sgd_falsePositiveRate, sgd_truePositiveRate, sgd_thresholds = sklearn.metrics.roc_curve(y_train, sgd_y_scores)

plt.figure(figsize=(6, 5))
plt.plot(sgd_falsePositiveRate, sgd_truePositiveRate, linewidth=2, color='black', label="SGD ROC curve")
plt.plot([0, 1], [0, 1], 'k:', label="Random classifier's ROC curve")
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import DetCurveDisplay, RocCurveDisplay

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 5))
RocCurveDisplay.from_estimator(sgd_classifier, x_test, y_test, ax=ax)
RocCurveDisplay.from_estimator(lr_classifier, x_test, y_test, ax=ax)
RocCurveDisplay.from_estimator(tree_classifier, x_test, y_test, ax=ax)
RocCurveDisplay.from_estimator(rf_classifier, x_test, y_test, ax=ax, color='k')